In [ ]:
# Image Segmentation with Watershed Algorithm
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
cd ../data

In [ ]:
# img = cv2.imread('smarties.png')
# img = cv2.imread('coins2.jpg')
img = cv2.imread('water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [ ]:
plt.imshow(thresh), plt.show()

In [ ]:
# noise removal opening
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
plt.imshow(opening), plt.show()

In [ ]:
# noise removal closing-opening
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 3)
opening = cv2.morphologyEx(closing,cv2.MORPH_OPEN,kernel, iterations = 2)
plt.imshow(opening), plt.show()

In [ ]:

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

In [ ]:
plt.subplot(141), plt.imshow(sure_bg)
plt.subplot(142), plt.imshow(sure_fg)
plt.subplot(143), plt.imshow(dist_transform)
plt.subplot(144), plt.imshow(unknown), 
plt.show()

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]

In [ ]:
plt.imshow(img), plt.show()

In [ ]:
cv2.imwrite('output.jpg',img)

In [ ]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)q
cv2.waitKey(0)
cv2.destroyAllWindows()